In [1]:
import os
import cv2
import matplotlib
import matplotlib.pyplot as plt
import h5py 

from tqdm import tqdm

from holobot.utils.files import *
from holobot.constants import *

In [2]:


def plot_line(X1, X2, Y1, Y2):
    plt.plot([X1, X2], [Y1, Y2])

class PlotHand2D():
    def __init__(self):

        # Thumb bound info
        self.thumb_bounds = None
        self.thumb_bounds_path = VR_DISPLAY_THUMB_BOUNDS_PATH
        self.bound_update_counter = 0
        self._check_thumb_bounds()

        # Checking image storage path
        # make_dir(os.path.join(PLOT_PATH))
        matplotlib.use('Agg')

        # Figure settings
        self.fig = plt.figure(figsize=(6, 6), dpi=60)

    def _check_thumb_bounds(self):
        if check_file(self.thumb_bounds_path):
            self.thumb_bounds = get_npz_data(self.thumb_bounds_path)

    def _set_limits(self):
        plt.axis([-0.12, 0.12, -0.02, 0.2])

    def _draw_thumb_bounds(self):
        for idx in range(VR_THUMB_BOUND_VERTICES):
            plot_line(
                self.thumb_bounds[idx][0], 
                self.thumb_bounds[(idx + 1) % VR_THUMB_BOUND_VERTICES][0], 
                self.thumb_bounds[idx][1], 
                self.thumb_bounds[(idx + 1) % VR_THUMB_BOUND_VERTICES][1]
            )
        
    def draw_hand(self, X, Y):
        plt.plot(X, Y, 'ro')

        if self.thumb_bounds is not None:
            self._draw_thumb_bounds()

        # Drawing connections fromn the wrist - 0
        for idx in OCULUS_JOINTS['metacarpals']:
            plot_line(X[0], X[idx], Y[0], Y[idx])

        # Drawing knuckle to knuckle connections and knuckle to finger connections
        for key in ['knuckles', 'thumb', 'index', 'middle', 'ring', 'pinky']:
            for idx in range(len(OCULUS_JOINTS[key]) - 1):
                plot_line(
                    X[OCULUS_JOINTS[key][idx]], 
                    X[OCULUS_JOINTS[key][idx + 1]], 
                    Y[OCULUS_JOINTS[key][idx]], 
                    Y[OCULUS_JOINTS[key][idx + 1]]
                )

    def draw(self, X, Y, fig_path=None):
        # Setting the plot limits
        self._set_limits()

        # Resetting the thumb bounds
        if self.bound_update_counter % 10 == 0:
            self._check_thumb_bounds()
            self.bound_update_counter = 0
        else:
            self.bound_update_counter += 1

        # Plotting the lines to visualize the hand
        self.draw_hand(X, Y)

        # Saving and obtaining the plot
        plt.savefig(fig_path)

        # Resetting and pausing the 3D plot
        # plt.pause(0.001) # This make the graph show up if matplotlib is in Tkinter mode
        plt.cla()

In [3]:

def dump_states(root, keypoint_indices, image_indices, view_num=0):
    # Make directory to dump the visualization
    pbar = tqdm(total=len(keypoint_indices))

    with h5py.File(os.path.join(root, 'keypoints.h5'), 'r') as f:
        hand_keypoints = f['transformed_hand_coords'][()]

    viz_dir = os.path.join(root, 'visualization')
    os.makedirs(viz_dir, exist_ok=True)

    # Get the hand plotter
    hand_plotter = PlotHand2D()
    
    video_path = os.path.join(root, f'cam_{view_num}_rgb_video.avi')
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_id = 0
    for i in range(len(keypoint_indices)):
        [_, keypoint_id], [_, image_id] = keypoint_indices[i], image_indices[i]
        while frame_id != image_id and success:
            # Find the frame that is equal to image_id
            success, image = vidcap.read()
            frame_id += 1
        dump_demo_state(
            frame_id = i,
            viz_dir = viz_dir,
            hand_plotter = hand_plotter,
            keypoint_values = hand_keypoints[keypoint_id,:],
            camera_img = image
        )

        pbar.update(1)

    pbar.close()

def dump_demo_state(frame_id, viz_dir, hand_plotter, keypoint_values, camera_img):
    hand_plotter.draw(
        keypoint_values[:, 0], keypoint_values[:, 1],
        fig_path=os.path.join(viz_dir, 'hand_coords.png'))

    hand_coord_img = cv2.imread(os.path.join(viz_dir, 'hand_coords.png'))
    height_scale = camera_img.shape[0] / hand_coord_img.shape[0]
    hand_coord_img = cv2.resize(
        hand_coord_img,
        (int(hand_coord_img.shape[1] * height_scale),
         int(hand_coord_img.shape[0] * height_scale))
    )
    total_img = cv2.hconcat([camera_img, hand_coord_img])

    img_name = 'state_{}.png'.format(str(frame_id).zfill(3))
    cv2.imwrite(os.path.join(viz_dir, img_name), total_img)



In [4]:
def load_indices(root):
    with open(os.path.join(root, 'keypoint_indices.pkl'), 'rb') as f:
        keypoint_indices = pickle.load(f)

    with open(os.path.join(root, 'image_indices.pkl'), 'rb') as f:
        image_indices = pickle.load(f)

    return keypoint_indices, image_indices

In [5]:
import glob
data_path = '/home/irmak/Workspace/Holo-Bot/extracted_data/human_sponge_flipping'
roots = glob.glob(f'{data_path}/demonstration_*')

view_num = 0
for root in roots:
    keypoint_indices, image_indices = load_indices(root)
    print('keypoint_indices: {}, image_indices: {}'.format(keypoint_indices, image_indices))
    dump_states(root, keypoint_indices, image_indices, view_num=view_num)


keypoint_indices: [[5, 0], [5, 2]], image_indices: [[5, 5], [5, 14]]


100%|██████████| 2/2 [00:00<00:00,  4.14it/s]


keypoint_indices: [[0, 0], [0, 19], [0, 37], [0, 55], [0, 73], [0, 91], [0, 109], [0, 127], [0, 145], [0, 163], [0, 181], [0, 198], [0, 216], [0, 234], [0, 252], [0, 270], [0, 288], [0, 306], [0, 324], [0, 342], [0, 360], [0, 378], [0, 396], [0, 414], [0, 432], [0, 450], [0, 468], [0, 486], [0, 504], [0, 522], [0, 540], [0, 558], [0, 576], [0, 594], [0, 610], [0, 629], [0, 647], [0, 665], [0, 683], [0, 701], [0, 719], [0, 737], [0, 755], [0, 773], [0, 791], [0, 809], [0, 827], [0, 845], [0, 863], [0, 881], [0, 898]], image_indices: [[0, 530], [0, 539], [0, 548], [0, 557], [0, 566], [0, 575], [0, 584], [0, 593], [0, 602], [0, 611], [0, 620], [0, 629], [0, 638], [0, 647], [0, 656], [0, 665], [0, 674], [0, 683], [0, 692], [0, 701], [0, 710], [0, 719], [0, 728], [0, 737], [0, 746], [0, 755], [0, 764], [0, 773], [0, 782], [0, 791], [0, 800], [0, 809], [0, 818], [0, 827], [0, 836], [0, 845], [0, 854], [0, 863], [0, 872], [0, 880], [0, 889], [0, 898], [0, 907], [0, 916], [0, 925], [0, 934], [

100%|██████████| 51/51 [00:07<00:00,  6.85it/s]


keypoint_indices: [[1, 0], [1, 18], [1, 36], [1, 54], [1, 72], [1, 90], [1, 108], [1, 126], [1, 144], [1, 162], [1, 180], [1, 198], [1, 216], [1, 234], [1, 252], [1, 270], [1, 288], [1, 306], [1, 324], [1, 342], [1, 360], [1, 377], [1, 395], [1, 413], [1, 431], [1, 449], [1, 467], [1, 485], [1, 503], [1, 521], [1, 539], [1, 557], [1, 575], [1, 593], [1, 611], [1, 629], [1, 647], [1, 665], [1, 683], [1, 701], [1, 719], [1, 737], [1, 755], [1, 773], [1, 791], [1, 807], [1, 825], [1, 843], [1, 861], [1, 879], [1, 897], [1, 915], [1, 933], [1, 951], [1, 969], [1, 987], [1, 999]], image_indices: [[1, 4], [1, 13], [1, 22], [1, 31], [1, 40], [1, 49], [1, 58], [1, 67], [1, 76], [1, 84], [1, 93], [1, 101], [1, 110], [1, 119], [1, 128], [1, 137], [1, 146], [1, 155], [1, 164], [1, 173], [1, 182], [1, 191], [1, 200], [1, 209], [1, 218], [1, 227], [1, 236], [1, 245], [1, 254], [1, 263], [1, 272], [1, 281], [1, 290], [1, 299], [1, 308], [1, 317], [1, 326], [1, 335], [1, 344], [1, 353], [1, 362], [1,

...

100%|██████████| 48/48 [00:06<00:00,  7.15it/s]


keypoint_indices: [[3, 0], [3, 18], [3, 36], [3, 54], [3, 72], [3, 90], [3, 108], [3, 126], [3, 144], [3, 162], [3, 180], [3, 196], [3, 214], [3, 232], [3, 248], [3, 266], [3, 284], [3, 302], [3, 320], [3, 338], [3, 356], [3, 374], [3, 392], [3, 410], [3, 428], [3, 446], [3, 464], [3, 482], [3, 500], [3, 518], [3, 536], [3, 554], [3, 572], [3, 590], [3, 608], [3, 624], [3, 642], [3, 660], [3, 678], [3, 696], [3, 714], [3, 732], [3, 750], [3, 768], [3, 786], [3, 804], [3, 822], [3, 832]], image_indices: [[3, 8], [3, 17], [3, 26], [3, 35], [3, 44], [3, 53], [3, 62], [3, 71], [3, 80], [3, 89], [3, 98], [3, 107], [3, 116], [3, 125], [3, 134], [3, 143], [3, 152], [3, 161], [3, 170], [3, 179], [3, 188], [3, 197], [3, 206], [3, 215], [3, 224], [3, 233], [3, 242], [3, 251], [3, 260], [3, 269], [3, 278], [3, 287], [3, 296], [3, 305], [3, 314], [3, 323], [3, 332], [3, 341], [3, 350], [3, 359], [3, 368], [3, 377], [3, 386], [3, 395], [3, 404], [3, 413], [3, 422], [3, 424]]


100%|██████████| 48/48 [00:07<00:00,  6.63it/s]


keypoint_indices: [[2, 0], [2, 18], [2, 36], [2, 54], [2, 72], [2, 90], [2, 108], [2, 126], [2, 144], [2, 162], [2, 180], [2, 198], [2, 216], [2, 234], [2, 252], [2, 270], [2, 288], [2, 306], [2, 324], [2, 342], [2, 360], [2, 378], [2, 396], [2, 414], [2, 432], [2, 450], [2, 468], [2, 486], [2, 500], [2, 518], [2, 536], [2, 554], [2, 572], [2, 590], [2, 608], [2, 626], [2, 644], [2, 662], [2, 680], [2, 698], [2, 716], [2, 734], [2, 752], [2, 770], [2, 788], [2, 806], [2, 824], [2, 842], [2, 860], [2, 878], [2, 896], [2, 914], [2, 931], [2, 947]], image_indices: [[2, 3], [2, 12], [2, 21], [2, 30], [2, 39], [2, 48], [2, 57], [2, 66], [2, 75], [2, 84], [2, 93], [2, 102], [2, 111], [2, 120], [2, 129], [2, 138], [2, 147], [2, 156], [2, 165], [2, 174], [2, 183], [2, 192], [2, 201], [2, 210], [2, 219], [2, 228], [2, 237], [2, 246], [2, 255], [2, 264], [2, 273], [2, 282], [2, 291], [2, 300], [2, 309], [2, 318], [2, 327], [2, 336], [2, 345], [2, 354], [2, 363], [2, 372], [2, 381], [2, 390], [2,

100%|██████████| 54/54 [00:07<00:00,  7.11it/s]


In [6]:
# Turn the images to a video and delete the directory
video_fps = 8
for root in roots:
    print('dumping video in root: {}'.format(root))
    video_path = os.path.join(root, f'visualization_{view_num}.mp4')
    if os.path.exists(video_path):
        os.remove(video_path)
    viz_dir = os.path.join(root, 'visualization')
    os.system('ffmpeg -r {} -i {}/%*.png -vf scale=2000x720,setsar=1:1 {}'.format(
        video_fps, # fps
        viz_dir,
        video_path
    ))


dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/human_sponge_flipping/demonstration_8


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-18)
  configuration: --prefix=/home/irmak/miniconda3/envs/tactile_learning --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/human_sponge_flipping/demonstration_1


Input #0, image2, from '/home/irmak/Workspace/Holo-Bot/extracted_data/human_sponge_flipping/demonstration_1/visualization/%*.png':
  Duration: 00:00:02.08, start: 0.000000, bitrate: N/A
  Stream #0:0: Video: png, rgba(pc), 360x360 [SAR 2362:2362 DAR 1:1], 25 fps, 25 tbr, 25 tbn
Stream mapping:
  Stream #0:0 -> #0:0 (png (native) -> h264 (libx264))
Press [q] to stop, [?] for help
[libx264 @ 0x55652387bc80] using SAR=1/1
[libx264 @ 0x55652387bc80] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x55652387bc80] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x55652387bc80] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 deci

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/human_sponge_flipping/demonstration_2


...

[libx264 @ 0x559e7431fd00] using SAR=1/1
[libx264 @ 0x559e7431fd00] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x559e7431fd00] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x559e7431fd00] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=8 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/human_sponge_flipping/dem

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/human_sponge_flipping/demonstration_5


[libx264 @ 0x560cee570d00] using SAR=1/1
[libx264 @ 0x560cee570d00] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x560cee570d00] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x560cee570d00] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=8 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/human_sponge_flipping/dem

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/human_sponge_flipping/demonstration_4


[libx264 @ 0x56459dabce00] using SAR=1/1
[libx264 @ 0x56459dabce00] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x56459dabce00] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x56459dabce00] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=8 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/human_sponge_flipping/dem

In [19]:
# Remove the img directory
import shutil
for root in roots:
    viz_dir = os.path.join(root, 'visualization')
    shutil.rmtree(viz_dir)
